# Query Electoral Database

This notebook shows how to query the SQLite database created by the pipeline and retrieve data as DataFrames for analysis.


## Setup


In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd() / 'src'))

from analytics.clean_votes import CleanVotesOrchestrator
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

%matplotlib inline


## 1. Initialize Orchestrator

The orchestrator provides easy access to the database.


In [ ]:
# Initialize with default database path
orchestrator = CleanVotesOrchestrator()

print(f"Database: {orchestrator.db_path}")
print(f"Database exists: {orchestrator.db_path.exists()}")


## 2. List All Available Elections

See what's in the database.


In [ ]:
# List all elections in database
elections = orchestrator.list_available_elections()

print(f"Total elections: {len(elections)}")
print(f"\nColumns: {elections.columns.tolist()}")
print(f"\nFirst 10 elections:")
elections.head(10)


In [ ]:
# See unique elections (not by entidad)
unique_elections = elections['election_name'].unique()
print(f"Unique elections: {len(unique_elections)}")
for election in unique_elections:
    count = len(elections[elections['election_name'] == election])
    print(f"  {election}: {count} entidades")


## 3. Load Specific Election Data

Load data for a specific election and state.


In [ ]:
# Load Presidential 2024 data for Aguascalientes (entidad_id=1)
df_ags = orchestrator.load_election_data(
    election_name='PRES_2024',
    entidad_id=1,
    as_geodataframe=False  # Load as regular DataFrame
)

print(f"Shape: {df_ags.shape}")
print(f"Columns: {df_ags.columns.tolist()[:10]}...")
df_ags.head()


## 4. Load with Geometry (GeoDataFrame)

Load data with geometry for spatial analysis and mapping.


In [ ]:
# Load with geometry
gdf_ags = orchestrator.load_election_data(
    election_name='PRES_2024',
    entidad_id=1,
    as_geodataframe=True  # Load as GeoDataFrame
)

print(f"Type: {type(gdf_ags)}")
print(f"Has geometry: {'geometry' in gdf_ags.columns}")
print(f"CRS: {gdf_ags.crs if hasattr(gdf_ags, 'crs') else 'N/A'}")

# Quick map
gdf_ags.explore(column='MORENA_PCT', cmap='Reds', legend=True)


## 5. Query Multiple States

Load and compare multiple states.


In [ ]:
# Load multiple states
states_to_compare = {
    1: 'Aguascalientes',
    9: 'CDMX',
    15: 'Estado de México',
    19: 'Nuevo León'
}

results = []

for entidad_id, name in states_to_compare.items():
    try:
        df = orchestrator.load_election_data('PRES_2024', entidad_id)
        results.append({
            'Entidad': name,
            'Sections': len(df),
            'Total Votes': df['TOTAL_VOTOS_SUM'].sum(),
            'MORENA %': df['MORENA_PCT'].mean(),
            'PAN %': df['PAN_PCT'].mean(),
            'PRI %': df['PRI_PCT'].mean()
        })
    except ValueError as e:
        print(f"No data for {name}: {e}")

comparison = pd.DataFrame(results)
comparison


## 6. Direct SQL Queries

You can also query directly using SQL if needed.


In [ ]:
import sqlite3

# Connect to database
db_path = orchestrator.db_path
conn = sqlite3.connect(db_path)

# Query election metadata
query = """
SELECT election_name, entidad_name, row_count, has_geometry, created_at
FROM election_metadata
WHERE election_name = 'PRES_2024'
ORDER BY entidad_id
"""

metadata_df = pd.read_sql_query(query, conn)
print(f"Found {len(metadata_df)} records for PRES_2024")
metadata_df.head(10)


In [ ]:
# Query specific election table directly
query = """
SELECT ID_ENTIDAD, SECCION, ENTIDAD, MORENA_PCT, PAN_PCT, TOTAL_VOTOS_SUM
FROM election_pres_2024_01
LIMIT 10
"""

direct_df = pd.read_sql_query(query, conn)
direct_df


In [ ]:
# Close connection
conn.close()


## 7. Combine Multiple Elections for Time Series Analysis


In [ ]:
# Compare MORENA performance across elections
elections_to_compare = ['PRES_2024', 'PRES_2018', 'DIP_FED_2021']
entidad_id = 9  # CDMX

temporal_data = []

for election in elections_to_compare:
    try:
        df = orchestrator.load_election_data(election, entidad_id)
        temporal_data.append({
            'Election': election,
            'MORENA %': df['MORENA_PCT'].mean(),
            'Sections': len(df),
            'Total Votes': df['TOTAL_VOTOS_SUM'].sum()
        })
    except ValueError:
        print(f"No data for {election}")

if temporal_data:
    temporal_df = pd.DataFrame(temporal_data)
    print("\nMORENA Performance Over Time (CDMX):")
    temporal_df


## 8. Export to Other Formats

Export data for use in other tools.


In [ ]:
# Load data
df_export = orchestrator.load_election_data('PRES_2024', entidad_id=1)

# Export to CSV
df_export.to_csv('data/insights/pres_2024_aguascalientes.csv', index=False)
print("✓ Exported to CSV")

# Export to Parquet (more efficient)
df_export.to_parquet('data/insights/pres_2024_aguascalientes.parquet', index=False)
print("✓ Exported to Parquet")

# If you have geometry, export to GeoJSON
gdf_export = orchestrator.load_election_data('PRES_2024', entidad_id=1, as_geodataframe=True)
if 'geometry' in gdf_export.columns:
    gdf_export.to_file('data/insights/pres_2024_aguascalientes.geojson', driver='GeoJSON')
    print("✓ Exported to GeoJSON")


## Summary

You can query the database using:

1. **`orchestrator.list_available_elections()`** - See what's available
2. **`orchestrator.load_election_data(election_name, entidad_id)`** - Load specific data
3. **Direct SQL queries** - For custom queries
4. **Export to CSV/Parquet/GeoJSON** - For use in other tools

All data is stored by (election, entidad) in separate tables for easy querying!
